# In Search of Contexts

In [ ]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search
from rapidfuzz import fuzz
import time

def get_targeted_proverb_context(proverb, threshold=80):
    # High-value targets for modern/classic quotes and memes
    targets = [
        "goodreads.com/quotes",
        "reddit.com",
        "brainyquote.com",
        "quotefancy.com",
        "pinterest.com", # Often has text in the 'alt' or 'description'
        "twitter.com"
    ]
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/119.0.0.0"}
    all_results = []

    print(f"🎯 Starting targeted search for: '{proverb}'")

    for domain in targets:
        # We use the 'site:' operator to narrow the focus
        query = f'"{proverb}" site:{domain}'
        print(f"🔎 Checking {domain}...")
        
        try:
            # We only need the top 2-3 results per site
            for url in search(query, num_results=3):
                time.sleep(1) # Be polite to avoid rate limits
                response = requests.get(url, headers=headers, timeout=10)
                
                if response.status_code != 200:
                    continue
                    
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Search across all text-heavy tags
                for tag in soup.find_all(['p', 'div', 'span', 'blockquote', 'li']):
                    text = " ".join(tag.get_text().split())
                    
                    if len(text) < len(proverb):
                        continue
                        
                    score = fuzz.partial_ratio(proverb.lower(), text.lower())
                    
                    if score >= threshold:
                        all_results.append({
                            "domain": domain,
                            "url": url,
                            "score": round(score, 2),
                            "text": text[:300]
                        })
                        break # Move to the next URL for this domain
                        
        except Exception as e:
            print(f"⚠️ Search failed for {domain}: {e}")
            continue
            
    return all_results

🎯 Starting targeted search for: 'The struggle is real, but so is the wifi signal'
🔎 Checking goodreads.com/quotes...
🔎 Checking reddit.com...
🔎 Checking brainyquote.com...
🔎 Checking quotefancy.com...
🔎 Checking pinterest.com...
🔎 Checking twitter.com...

🤷 No text-based matches found in the targeted databases.


In [2]:
# Test it
proverb = "The internet is forever so be careful what you post"
matches = get_targeted_proverb_context(proverb)

if matches:
    print(f"\n✨ Found {len(matches)} actual text matches:")
    for m in matches:
        print(f"[{m['score']}%] Source: {m['url']}")
        print(f"Text: {m['text']}\n")
else:
    print("\n🤷 No text-based matches found in the targeted databases.")

🎯 Starting targeted search for: 'The internet is forever so be careful what you post'
🔎 Checking goodreads.com/quotes...
🔎 Checking reddit.com...
🔎 Checking brainyquote.com...
🔎 Checking quotefancy.com...
🔎 Checking pinterest.com...
🔎 Checking twitter.com...

🤷 No text-based matches found in the targeted databases.
